This notebook was almost entirely designed by Elise Olson. It does a lot of cleaning for the Washington Department of Ecology nutrient and chlorophyll bottle collections that occured for 2018 and 2019. It is very important to understand that these chlorophyll and nutrient observations are independent of the ones that are loaded using WADE_Pickle_Loader and Chlbot_Pickle_Loader.  

In [39]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import datetime as dt
from salishsea_tools import evaltools as et

In [36]:
modelversion='nowcast-green.201905'
PATH= '/results2/SalishSea/nowcast-green.201905/'
datadir='/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology'

In [2]:
# read in stored station info (links station names to locations)
dir0 = '/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology/'
sta_df = pd.read_pickle(dir0 + 'sta_df.p')

In [3]:
bottle_fn = dir0 + 'raw/ParkerMacCready2019CTDDataFeb2020.xlsx'
sheet_name = '2018-2019NutrientData'
sheet_chl = '2018-2019ChlaLabData'
bot = pd.read_excel(bottle_fn, sheet_name=sheet_name,engine='openpyxl')
chl = pd.read_excel(bottle_fn, sheet_name=sheet_chl, engine='openpyxl')

In [4]:
bot.dropna(how='all',inplace=True) # drop rows where all values are NaN
chl.dropna(how='all',inplace=True)
bot.dropna(subset=['Sampling Depth'],inplace=True) # drop rows with NaN sampling depths because nominal depths do not appear reliable
chl.dropna(subset=['Sampling Depth'],inplace=True)
# drop rows with no useful data:
bot.dropna(how='all',subset=['NH4_Lab','NO2_Lab','NO3_Lab','PO4_Lab','SiOH4_Lab'],inplace=True)
chl.dropna(how='all',subset=['Chla_Lab'],inplace=True)

In [5]:
# drop rows that appear to have misaligned Sampling depths
#Date	Station	Niskin	Nomdepth	Sampling Depth	CTD Cast Rep	Chla_Lab	Chla_QC	
    #2019-02-26	DNA001	12.0	30	1.136	1.0	1.9216	2.0	
chl.drop(chl.loc[(chl.Date==dt.datetime(2019,2,26))&(chl.Station=='DNA001')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==1.9216)].index,inplace=True)
# also remove from chl: 
#       2018-02-01  HCB010     9.0       30          10.545 sample is probably actually 30 m based on nuts
#       2018-03-09  CRR001     9.0       30         101.554 "
chl.drop(chl.loc[(chl.Date==dt.datetime(2018,2,1))&(chl.Station=='HCB010')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==10.545)].index,inplace=True)
chl.drop(chl.loc[(chl.Date==dt.datetime(2018,3,9))&(chl.Station=='CRR001')&\
                 (chl.Nomdepth==30)&(chl['Sampling Depth']==101.554)].index,inplace=True)
#remove from chl and nuts:
#       Date:2019-10-30 Station: ADM001  all 3 values (Nom depth 0,10,30 but sampling depth all ~126.7)
chl.drop(chl.loc[(chl.Date==dt.datetime(2019,10,30))&(chl.Station=='ADM001')].index,inplace=True)
bot.drop(chl.loc[(chl.Date==dt.datetime(2019,10,30))&(chl.Station=='ADM001')].index,inplace=True)

In [6]:
# average over rows having identical  (Date,Station,Niskin,Sampling Depth) [replicates]
# ignore Nomdepth because it seems to not always be accurate
chl2=pd.DataFrame(chl.groupby(['Date','Station','Niskin','Sampling Depth'],as_index=False).mean())

In [7]:
chl2

,Date,Station,Niskin,Sampling Depth,CTD Cast Rep,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber
0,2018-01-10,PSS019,9.0,31.196,1.0,0.0710,2.0,3.0,1.0
1,2018-01-10,PSS019,11.0,11.404,1.0,0.1960,2.0,3.0,1.0
2,2018-01-10,PSS019,12.0,1.187,1.0,0.6673,2.0,3.0,1.0
3,2018-01-10,SAR003,9.0,31.057,1.0,0.0530,2.0,3.0,1.0
4,2018-01-10,SAR003,11.0,11.146,1.0,0.2097,2.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...
1787,2019-12-19,NSQ002,9.0,30.676,1.0,0.3154,2.0,2.0,1.0
1788,2019-12-19,NSQ002,10.0,10.871,1.0,0.3640,2.0,2.0,1.0
1789,2019-12-19,NSQ002,12.0,1.214,1.0,0.4397,2.0,2.0,1.0
1790,2019-12-19,OAK004,11.0,11.106,1.0,0.5765,2.0,2.0,2.0


In [8]:
bot2=pd.DataFrame(bot.groupby(['Date','Station','Niskin','Sampling Depth'],as_index=False).mean())

In [9]:
allbot = pd.merge(left=bot2,right=chl2,how='outer',
                    left_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth'],
                    right_on = ['Date','Station','Niskin','CTD Cast Rep','Sampling Depth'])
print(f'len(allbot):{len(allbot)}, len(chl2):{len(chl2)}, len(bot2):{len(bot2)}')

len(allbot):2357, len(chl2):1792, len(bot2):2348


In [10]:
nutNoChl=len(allbot.loc[(~pd.isnull(allbot.NH4_Lab))&(pd.isnull(allbot.Chla_Lab))])
chlNoNut=len(allbot.loc[(pd.isnull(allbot.NH4_Lab))&(~pd.isnull(allbot.Chla_Lab))])
print(nutNoChl,chlNoNut)

565 9


In [11]:
# if these equal the total length, things are making sense:
print(len(bot2)+chlNoNut,len(chl2)+nutNoChl)

2357 2357


In [12]:
allbot.keys()

Index(['Date', 'Station', 'Niskin', 'Sampling Depth', 'CTD Cast Rep',
       'NH4_Lab', 'NH4_QC', 'NH4_QA', 'NH4_SampleFieldReplicateNumber',
       'NO2_Lab', 'NO2_QC', 'NO2_QA', 'NO2_SampleFieldReplicateNumber',
       'NO3_Lab', 'NO3_QC', 'NO3_QA', 'NO3_SampleFieldReplicateNumber',
       'PO4_Lab', 'PO4_QC', 'PO4_QA', 'PO4_SampleFieldReplicateNumber',
       'SiOH4_Lab', 'SiOH4_QC', 'SiOH4_QA', 'SiOH4_SampleFieldReplicateNumber',
       'Unnamed: 36', 'Chla_Lab', 'Chla_QC', 'Chla_QA',
       'Chla_SampleFieldReplicateNumber'],
      dtype='object')

#### Now add date/times

In [13]:
dfTime=pd.read_excel('/ocean/eolson/MEOPAR/obs/WADE/WDE_Data/OlsonSuchyAllen_UBC_PDR_P003790-010721.xlsx',
                    engine='openpyxl',sheet_name='EventDateTime')

In [14]:
## duplicate Station/Date entries with different times seem to be always within a couple of hours, 
# so just take the first (next cell)
test=dfTime.groupby(['FlightDate','SiteCode'])['TimeDown \n(Local - PST or PDT)'].count()
for date, loc in test[test>1].index:
    print(dfTime.loc[(dfTime.FlightDate==date)&(dfTime.SiteCode==loc),['FlightDate','SiteCode','TimeDown \n(Local - PST or PDT)']])

    FlightDate SiteCode TimeDown \n(Local - PST or PDT)
590 2001-07-09   DNA001                        17:09:00
591 2001-07-09   DNA001                        17:09:00
    FlightDate SiteCode TimeDown \n(Local - PST or PDT)
948 2003-06-25   BUD005                        17:00:00
949 2003-06-25   BUD005                        17:00:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1362 2005-08-16   BUD005                        10:43:00
1363 2005-08-16   BUD005                        10:43:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1429 2005-11-07   BUD005                        11:55:00
1430 2005-11-07   BUD005                        11:55:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
1446 2005-12-05   BUD005                        11:58:00
1447 2005-12-05   BUD005                        11:58:00
     FlightDate SiteCode TimeDown \n(Local - PST or PDT)
2104 2008-08-21   SJF000                        09:59:00
2105 2008-08-21   SJF000             

In [15]:
# drop duplicate rows
dfTime.drop_duplicates(subset=['FlightDate','SiteCode'],keep='first',inplace=True)
print(dfTime.keys())

Index(['FlightYear', 'FlightMonth', 'FlightDate', 'SiteCode', 'Sampled',
       'TimeDown \n(Local - PST or PDT)', 'FieldComment'],
      dtype='object')


In [16]:
dfTime['dtPac']=[dt.datetime.combine(idate, itime) for idate, itime \
         in zip(dfTime['FlightDate'],dfTime['TimeDown \n(Local - PST or PDT)'])]
dfTime['dtUTC']=[et.pac_to_utc(ii) for ii in dfTime['dtPac']]

In [17]:
allbot['dtUTC']=np.nan # create column and set all values to nan

In [18]:
# use loop to set dtUTC values, where available (where dates are unavailable, left as NaN for now)
for ind, row in allbot.iterrows():
    ix=(dfTime.FlightDate==row['Date'])&(dfTime.SiteCode==row['Station'])
    if np.sum(ix)==1:
        idate,itime=dfTime.loc[ix,['FlightDate','TimeDown \n(Local - PST or PDT)']].values[0]
        allbot.loc[ind,['dtUTC']]=et.pac_to_utc(dt.datetime.combine(idate,itime))

In [19]:
# where no time is provided, set time to midday Pacific time = ~ 20:00 UTC for now
# (most sampling takes place during the day)
# accurate times will be provided at a later date
# the code below takes advantage of all elements in 'Date' having a time component 
# set to midnight
allbot['dtUTC']=[iiD+dt.timedelta(hours=20) if pd.isnull(iiU) \
                    else iiU for iiU,iiD in \
                    zip(allbot['dtUTC'],allbot['Date'])]

#### Now add lat/lons

In [20]:
# PROCESS STATION LOCATION INFO (based on Parker's code)
sta_fn='/ocean/eolson/MEOPAR/obs/WADE/WDE_Data/OlsonSuchyAllen_UBC_PDR_P003790-010721.xlsx'
sheetname='Site Info'
sta_df =pd.read_excel(sta_fn,engine='openpyxl',sheet_name=sheetname)
sta_df.dropna(how='any',subset=['Lat_NAD83 (deg / dec_min)','Long_NAD83 (deg / dec_min)','Station'],inplace=True)
sta_df = sta_df.set_index('Station')
# get locations in decimal degrees
for sta in sta_df.index:
    lat_str = sta_df.loc[sta, 'Lat_NAD83 (deg / dec_min)']
    lat_deg = float(lat_str.split()[0]) + float(lat_str.split()[1])/60
    sta_df.loc[sta,'Lat'] = lat_deg
    #
    lon_str = sta_df.loc[sta, 'Long_NAD83 (deg / dec_min)']
    lon_deg = float(lon_str.split()[0]) + float(lon_str.split()[1])/60
    sta_df.loc[sta,'Lon'] = -lon_deg    
sta_df.pop('Lat_NAD83 (deg / dec_min)');
sta_df.pop('Long_NAD83 (deg / dec_min)');

In [21]:
allbotFinal=pd.merge(left=sta_df,right=allbot,how='right',
                 left_on='Station',right_on='Station')

In [22]:
allbotFinal

,Station,Desig,Descrip,Basin,*Max_Depth,Lat,Lon,Date,Niskin,Sampling Depth,...,SiOH4_Lab,SiOH4_QC,SiOH4_QA,SiOH4_SampleFieldReplicateNumber,Unnamed: 36,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber,dtUTC
0,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,3,26.820,...,54.8964,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-01-10 20:00:00
1,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,11,11.252,...,55.2832,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-01-10 20:00:00
2,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-10,12,1.336,...,56.8617,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-01-10 20:00:00
3,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07,11,10.865,...,47.2309,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-03-07 20:00:00
4,HCB013,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-07,12,1.170,...,44.2977,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-03-07 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-06-04,12,1.220,...,25.4888,2.0,2.0,1.0,NaN,6.2169,2.0,2.0,1.0,2019-06-04 20:00:00
2353,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-07-02,11,5.963,...,21.8532,2.0,2.0,1.0,NaN,6.7821,2.0,2.0,1.0,2019-07-02 20:00:00
2354,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-07-02,12,1.469,...,21.4286,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,2019-07-02 20:00:00
2355,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644,-123.993,2019-09-11,11,6.032,...,30.8068,2.0,2.0,1.0,NaN,9.7210,2.0,2.0,1.0,2019-09-11 20:00:00


In [23]:
np.unique(allbotFinal.loc[np.isnan(allbotFinal.Lat),['Station']])

array(['BLL040', 'HCB013'], dtype=object)

In [24]:
# There are two station designations that are not pressent in the station list. Drop them.
allbotFinal.dropna(how='any',subset=['Lat','Lon'],inplace=True)

In [25]:
# rename Sample Depth to Z
allbotFinal.rename(columns={'Sampling Depth':'Z'},inplace=True)

In [26]:
allbotFinal

,Station,Desig,Descrip,Basin,*Max_Depth,Lat,Lon,Date,Niskin,Z,...,SiOH4_Lab,SiOH4_QC,SiOH4_QA,SiOH4_SampleFieldReplicateNumber,Unnamed: 36,Chla_Lab,Chla_QC,Chla_QA,Chla_SampleFieldReplicateNumber,dtUTC
22,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,3,98.380,...,62.2388,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-01-10 20:34:00
23,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,9,31.196,...,56.2757,2.0,3.0,1.0,NaN,0.0710,2.0,3.0,1.0,2018-01-10 20:34:00
24,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,11,11.404,...,59.8053,2.0,3.0,1.0,NaN,0.1960,2.0,3.0,1.0,2018-01-10 20:34:00
25,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-01-10,12,1.187,...,63.2084,2.0,3.0,1.0,NaN,0.6673,2.0,3.0,1.0,2018-01-10 20:34:00
26,PSS019,C,Possession Sound - Gedney Island,Whidbey Basin,107.0,48.010927,-122.30125,2018-03-07,3,86.010,...,62.0471,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2018-03-07 21:27:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-06-04,12,1.220,...,25.4888,2.0,2.0,1.0,NaN,6.2169,2.0,2.0,1.0,2019-06-04 20:00:00
2353,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-07-02,11,5.963,...,21.8532,2.0,2.0,1.0,NaN,6.7821,2.0,2.0,1.0,2019-07-02 20:00:00
2354,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-07-02,12,1.469,...,21.4286,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,2019-07-02 20:00:00
2355,WPA113,"C,M,T",Willapa Bay - Bay Center (red nun 2) - MT008,Willapa Bay,10.0,46.644000,-123.99300,2019-09-11,11,6.032,...,30.8068,2.0,2.0,1.0,NaN,9.7210,2.0,2.0,1.0,2019-09-11 20:00:00


In [32]:
allbotFinal['NO23']=allbotFinal['NO3_Lab']+allbotFinal['NO2_Lab'] # the model does not distinguish between NO2 and NO3
allbotFinal['Amm']=allbotFinal['NH4_Lab']
allbotFinal['Si']=allbotFinal['SiOH4_Lab']
allbotFinal['Year']=[ii.year for ii in allbotFinal['dtUTC']]
allbotFinal['YD']=et.datetimeToYD(allbotFinal['dtUTC'])

In [34]:
start_date = dt.datetime(2018,1,1)
end_date = dt.datetime(2019,12,31)
flen=1 # number of days per model output file. always 1 for 201905 and 201812 model runs
namfmt='nowcast' # for 201905 and 201812 model runs, this should always be 'nowcast'
filemap={'diatoms':'ptrc_T','ciliates':'ptrc_T','flagellates':'ptrc_T','votemper':'grid_T',
         'vosaline':'grid_T','nitrate':'ptrc_T','silicon':'ptrc_T','ammonium':'ptrc_T'}
fdict={'ptrc_T':1,'grid_T':1}

In [37]:
data=et.matchData(allbotFinal,filemap,fdict,start_date,end_date,'nowcast',PATH,1,quiet=False);

(Lat,Lon)= 46.453155 -124.00960333333333  not matched to domain
(Lat,Lon)= 46.463155 -123.94126833333334  not matched to domain
(Lat,Lon)= 46.54537666666667 -123.98016166666666  not matched to domain
(Lat,Lon)= 46.644 -123.993  not matched to domain
(Lat,Lon)= 46.68676333333333 -123.9735  not matched to domain
(Lat,Lon)= 46.703986666666665 -123.837385  not matched to domain
(Lat,Lon)= 46.937313333333336 -123.91322333333333  not matched to domain
(Lat,Lon)= 46.953421666666664 -124.09295  not matched to domain
(Lat,Lon)= 47.21342666666666 -123.07765  not matched to domain


In [40]:
##### Saving data as Pickle files
saveloc='/ocean/kflanaga/MEOPAR/savedData'
with open(os.path.join(saveloc,f'data_nutchl_{modelversion}_2018-2019.pkl'),'wb') as hh:
    pickle.dump(data,hh)